In [1]:
suppressPackageStartupMessages({
    require(Seurat)
    require(dplyr)
})

In [2]:
Hsap <- readRDS('Hsap.wb.iter_cluster_annotated.rds')
Mmus <- readRDS('Mmus.wb.iter_cluster_annotated.rds')
Pvit <- readRDS('Pvit.wb.iter_cluster_annotated.rds')
Pmar <- readRDS('Pmar.wb.iter_cluster_annotated.rds')

In [3]:
species <- c('Hsap', 'Mmus', 'Pvit', 'Pmar')

In [4]:
combination <- combn(species, 2)

In [5]:
path <- '/mnt/data01/yuanzhen/01.Vertebrate_cell_evo/02.gene_relationships/run4/results/Ortho_pipeline/OrthoFinder/RBH_results/'

for(i in 1:ncol(combination)){
    tmp <- read.delim(paste0(path, '/RBH.', combination[1, i], '.', combination[2, i], '.tsv'), header = T)
    names <- paste0('RBH_', combination[1, i], '.', combination[2, i])
    assign(names, tmp)
}

In [6]:
Amniotes_orthologs <- RBH_Hsap.Mmus %>% full_join(RBH_Mmus.Pvit, by = "Mmus") %>% 
    full_join(RBH_Hsap.Pvit, by = c("Pvit", "Hsap"))

In [7]:
Vertebrate_orthologs <- Amniotes_orthologs %>% full_join(RBH_Hsap.Pmar, by = "Hsap") %>%
    full_join(RBH_Mmus.Pmar, by = c("Pmar","Mmus")) %>% full_join(RBH_Pvit.Pmar, by = c("Pmar","Pvit"))

In [8]:
gene_bg <- readRDS('/mnt/data01/yuanzhen/01.Vertebrate_cell_evo/04.ohno_para_significance/0bin/gene_bg.rds')

In [9]:
# make sure every genes in 1-1-1-1 orthologs are expressed (included) in atlases
Vertebrate_orthologs <- Vertebrate_orthologs %>% filter(Hsap %in% gene_bg[gene_bg$species == 'Hsap', 'gene'] & 
                                                        Mmus %in% gene_bg[gene_bg$species == 'Mmus', 'gene'] &
                                                        Pvit %in% gene_bg[gene_bg$species == 'Pvit', 'gene'] & 
                                                        Pmar %in% gene_bg[gene_bg$species == 'Pmar', 'gene'])
Vertebrate_orthologs <- Vertebrate_orthologs[complete.cases(Vertebrate_orthologs),]

Amniotes_orthologs <- Amniotes_orthologs %>% filter(Hsap %in% gene_bg[gene_bg$species == 'Hsap', 'gene'] & 
                                                        Mmus %in% gene_bg[gene_bg$species == 'Mmus', 'gene'] &
                                                        Pvit %in% gene_bg[gene_bg$species == 'Pvit', 'gene'])
Amniotes_orthologs <- Amniotes_orthologs[complete.cases(Amniotes_orthologs),]

In [10]:
meta <- c('DonorID','Refined family', 'Refined subtype', 'Species')

In [11]:
subset_rename_data <- function(obj, table, species_from, species_to){
    raw <- GetAssayData(obj, layer = "counts")
    raw <- raw[table[,species_from], ]
    rownames(raw) <- table[,species_to]
    
    normalised <- GetAssayData(obj, layer = "data")
    normalised <- normalised[table[,species_from], ]
    rownames(normalised) <- table[, species_to]
    
    seurat_object <- CreateSeuratObject(counts = raw, meta.data = obj@meta.data[,meta])
    seurat_object[["RNA"]] <- CreateAssay5Object(counts = raw, data = normalised)
    return(seurat_object)
}

In [13]:
write.table(Amniotes_orthologs, file = "Amniote.1-1orthologs.txt", col.names = F, row.names = F, quote = F, sep = '\t')
write.table(Vertebrate_orthologs, file = "Vertebrate.1-1orthologs.txt", col.names = F, row.names = F, quote = F, sep = '\t')

In [14]:
Hsap_renamed <- subset_rename_data(Hsap, Amniotes_orthologs, 'Hsap', 'Mmus')
Mmus_renamed <- subset_rename_data(Mmus, Amniotes_orthologs, 'Mmus', 'Mmus')
Pvit_renamed <- subset_rename_data(Pvit, Amniotes_orthologs, 'Pvit', 'Mmus')

amniote_obj <- merge(Hsap_renamed, y = c(Mmus_renamed, Pvit_renamed))
Idents(amniote_obj) <- "Refined.family"
amniote_obj <- JoinLayers(amniote_obj, overwrite = TRUE)
saveRDS(amniote_obj, file = "Amniote.1-1orthologs.rds")

In [15]:
Hsap_renamed <- subset_rename_data(Hsap, Vertebrate_orthologs, 'Hsap', 'Mmus')
Mmus_renamed <- subset_rename_data(Mmus, Vertebrate_orthologs, 'Mmus', 'Mmus')
Pvit_renamed <- subset_rename_data(Pvit, Vertebrate_orthologs, 'Pvit', 'Mmus')
Pmar_renamed <- subset_rename_data(Pmar, Vertebrate_orthologs, 'Pmar', 'Mmus')

vertebrate_obj <- merge(Hsap_renamed, y = c(Mmus_renamed, Pvit_renamed, Pmar_renamed))
Idents(vertebrate_obj) <- "Refined.family"
vertebrate_obj <- JoinLayers(vertebrate_obj, overwrite = TRUE)
saveRDS(vertebrate_obj, file = "Vertebrate.1-1orthologs.rds")